In [65]:
import os
import re
from camel_tools.utils import normalize
from ibm_watsonx_ai.foundation_models import Model
import numpy as np
from camel_tools.utils import normalize
from pyarabic.araby import strip_tashkeel, strip_tatweel
import camel_tools.utils.normalize as normalize
from camel_tools.tokenizers.word import simple_word_tokenize
from rouge import Rouge
import json
from typing import List, Dict, Union
from collections import Counter
import time
import logging
from tqdm import tqdm


In [66]:

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey": os.environ.get("APIKEY"),
	}


In [67]:
model_id = "sdaia/allam-1-13b-instruct"


In [68]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")

In [69]:
def sample_model_parameters(decoding_method, max_new_tokens, temperature, repetition_penalty, top_p, top_k):
    return {
        "decoding_method": decoding_method,
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "repetition_penalty": repetition_penalty, 
        "top_p": top_p,
        "top_k": top_k,
        }

def greedy_model_parameters(decoding_method, max_new_tokens, repetition_penalty):
    return {
        "decoding_method": decoding_method,
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": repetition_penalty, 
        }

In [70]:
decoding_sample = Model(
	model_id = model_id,
	params = sample_model_parameters("sample",500, 0.7, 1.1, 1.0 ,60),
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-10-01&project_id=ef52a63f-03a1-472c-b1ac-25e87560e06f&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


In [71]:
decoding_greedy = Model(
	model_id = model_id,
	params = greedy_model_parameters("greedy",500, 1.1),
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)

INFO:ibm_watsonx_ai.client:Client successfully initialized
INFO:ibm_watsonx_ai.wml_resource:Successfully finished Get available foundation models for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-10-01&project_id=ef52a63f-03a1-472c-b1ac-25e87560e06f&filters=function_text_generation%2C%21lifecycle_withdrawn%3Aand&limit=200'


In [72]:
text1 = """
السلام عليكم ورحمة الله وبركاته أعزائي طلاب وطالبات الصف الخامس أهلا بكم من جديد في درس من دروس مادتنا الدراسات الاجتماعية معي معلمتكم الأستاذ العنود الاعتيبي برفقة زميلنا الأستاذ عماد الذويب في لغة الإشارة فأهلا بكم بسم الله نبدأ درسنا الثاني الوحدة الأولى الخلفاء الراشدون ماذا سوف أتعلم في هذه الوحدة يا ترى صفات الخلفاء الراشدين أعمال الخلفاء الراشدين نشر الإسلام في عهد الخلفاء الراشدين نتائج الفتن بداية درس الأولى الخلفاء الراشدين تليها دروس الوحدة الخليفة أبو بكر الصديق الخليفة عمر بن الخطاب الخليفة عثمان بن عفان الخليفة علي بن أبي طالب في نهاية كل وحدة يوجد لدينا تقويم شامل للوحدة محاور درسنا لهذا اليوم الدرس الأول من دروس مادة الدراسات الاجتماعية الخلفاء الراشدين صفات الخلفاء الراشدين أعمال الخلفاء الراشدين تسلسل الخط الزمني للخلافة الراشدة أهم معارك الخلفاء الراشدين مع الروم والفرس هذه هي محاور درسنا لهذا اليوم أهداف درسنا لهذا اليوم أن يعدد الطلبة والطالبات صفات الخلفاء الراشدين أن يذكر الطلبة والطالبات أعمال الخلفاء الراشدين أن يتتبع الطلبة والطالبات تسلسل الخط الزمني للخلافة الراشدة.
 """
text2 = """
أيضا تذكير محاور درسنا لهذا اليوم أن يكون هناك موضوع في عدة مدارس يقوم فيها الطلاب بتعلم وتدريس الصفات المختلفة، كذلك نوضح الطلبة والطالبات أهم معارك الخلفاء الراشدين مع بلاد الروم والفرس. أهداف درسنا لهذا اليوم تشمل فهم بداية الخلافة الراشدة مع تولي أبو بكر الصديق، ما حدث قبل وفاة النبي صلى الله عليه وسلم عندما نزلت آخر آية في القرآن الكريم. بكى أبو بكر الصديق رضي الله عنه عند نزول الآية لأنها كانت نعيًا للرسول صلى الله عليه وسلم. تفاجأ المسلمون بوفاة النبي واحتاجوا لخليفة لقيادتهم، فتولى الخلفاء الراشدين شؤون المسلمين بعده. سنتعرف على فضائل الخلفاء الراشدين ومواقفهم الهامة في درس اليوم.
"""
text3 = """
ثم نأتي إلى حديث الرسول صلى الله عليه وسلم عليكم بسنته وسنة الخلفاء الراشدين المهديين، وقال الله تعالى بعد النبي أربعة خلفاء عبر البيعة الشرعية. كانت خلافة أبي بكر الصديق بعد اجتماع المسلمين في سقيفة بني ساعدة، ثم اختيار عمر بن الخطاب بتزكية من أبي بكر، ثم عثمان بن عفان باختيار الصحابة، وعلي بن أبي طالب بإجماع الصحابة. الخلفاء الراشدون تولوا بعد النبي وسموا راشدين لعدلهم وخدمتهم للإسلام. سنتعرف على تسلسل الخلفاء الراشدين وتواريخهم، بدءاً من أبي بكر ثم عمر ثم عثمان وأخيراً علي رضي الله عنهم جميعاً. كل منهم استمر بفترة حكم مؤثرة في تاريخ الإسلام.

"""
text4= """
فضائل الخلفاء الراشدين تشمل كونهم من أوائل من أسلموا، ومن العشرة المبشرين بالجنة، ومن أقوى الصحابة في إدارة شؤون المسلمين. ننطلق إلى نشاط طلابي يتضمن تذكر تواريخ نهاية عهد كل خليفة. انتهت خلافة أبي بكر عام 13 هـ، وعمر بن الخطاب عام 23 هـ، وعثمان بن عفان عام 35 هـ، وعلي بن أبي طالب عام 40 هـ. تميزت هذه الفترات بالعدل والمساواة، وكانت معاركهم ضد الروم والفرس ناجحة ومؤثرة. الفتوحات في عهدهم امتدت إلى الشام، العراق، فارس، وشمال أفريقيا، وحدثت معارك بارزة كالقادسية واليرموك ضد الفرس والروم.

"""
text5= """
بعض من أبرز معارك الخلفاء الراشدين كانت معركة أجنادين، معركة ذات السلاسل، ومعركة اليرموك. تأسست الدولة الإسلامية على العدل والمساواة بقيادة الخلفاء الراشدين، الذين كانوا من بين أفضل الصحابة وأشدهم إخلاصًا للدين. يعد حكم الخلفاء الراشدين امتداداً للدولة الإسلامية كما كانت في عهد النبي، وانتشرت الفتوحات إلى خارج شبه الجزيرة العربية حيث توسعت الدولة الإسلامية إلى شمال أفريقيا، النوبة، وبلاد فارس. أهداف الدرس شملت تعريف الطلبة بأبرز معارك الخلفاء وفضائلهم، وكذلك واجب التفكير في أهمية العدل والإخلاص في إدارة شؤون المسلمين.
"""

text6= """
أصول علوم الأحياء الحديثة ومنهجها في دراسة الطبيعة تعود إلى اليونان القديمة، حيث كان أبقراط بمثابة مؤسس علم الطب، بالإضافة إلى مُساهمة أرسطو الكبيرة في تطوير علم الأحياء، حيث كان لكتبه التي أظهر فيها ميوله للطبيعة أهميةٌ خاصةٌ مثل كتاب "تاريخ الحيوانات"، تبع ذلك أعمالٌ أكثر تجريبية ركّزت على السببية البيولوجية وتنوع الحياة.
وكتب العالم ثيوفراستوس بعد ذلك سلسلة من الكتب في علم النبات اعتُبرت الأهم من نوعها في هذا العلم في العصور القديمة حتى العصور الوسطى. وقد ظهر مصطلح علم الأحياء للمرة الأولى عام 1736 عندما استخدمه كارلوس لينيوس في أحد كتبه، ثم دخل هذا المُصطلح حيِّز الاستخدام الحديث في أطروحةٍ من تأليف العالم الألماني غوتفريد راينولد تريفيرانوس، الذي تحدث عن أشكال الحياة ومظاهرها المُختلفة، والظروف والقوانين التي تحدث بموجبها هذه الظواهر، والأمور والأسباب التي أثرت فيها، وسمها باسم علم الأحياء.
"""
text7= """
وكتب العالم ثيوفراستوس بعد ذلك سلسلة من الكتب في علم النبات اعتُبرت الأهم من نوعها في هذا العلم في العصور القديمة حتى العصور الوسطى. وقد ظهر مصطلح علم الأحياء للمرة الأولى عام 1736 عندما استخدمه كارلوس لينيوس في أحد كتبه، ثم دخل هذا المُصطلح حيِّز الاستخدام الحديث في أطروحةٍ من تأليف العالم الألماني غوتفريد راينولد تريفيرانوس، الذي تحدث عن أشكال الحياة ومظاهرها المُختلفة، والظروف والقوانين التي تحدث بموجبها هذه الظواهر، والأمور والأسباب التي أثرت فيها، وسمها باسم علم الأحياء.
وقفز علم الأحياء قفزةً كبيرةً عندما قام أنطوني فان ليفينهوك بتطوير المجهر، حيث أدى ذلك إلى اكتشاف الحيوانات المنوية والبكتيريا ومُختلف الكائنات المجهرية. كما لعب العالم الهولندي جان سوامردام دوراً محورياً في تطوير علم الحشرات وساعد في إرساء التقنيات الأساسية في الترشيح والتلوين المجهري. كما كان للتقدم في الدراسات المهاجرية أثرٌ عميقٌ في تفكير الأحيائيين، فأشار عددٌ من علماء الأحياء إلى الأهمية المركزية للخلية منذ مطلع القرن التاسع عشر.
"""

In [73]:
def preprocess_arabic_text(text):
    text = strip_tatweel(text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[A-Za-z]', '', text)
    return text

In [74]:
preprocessedText1 = preprocess_arabic_text(text1)
preprocessedText2 = preprocess_arabic_text(text2)
preprocessedText3 = preprocess_arabic_text(text3)
preprocessedText4 = preprocess_arabic_text(text4)
preprocessedText5 = preprocess_arabic_text(text5)
preprocessedText6 = preprocess_arabic_text(text6)
preprocessedText7 = preprocess_arabic_text(text7)

In [75]:
def summarizationPrompt(text):
    prompt = f""" 
    You are a summarization tool. 
    Condense the text within the <context> tag by extracting only the main ideas and key supporting details. 
    Ensure the summary covers a fuller range of important points to provide a clearer picture of the content. 
    Avoid minor details or redundant information.
    Focus exclusively on the main ideas, primary learning objectives, and significant examples or evidence. 
    Avoid including or summarizing any lesson outline, agenda, or structural elements.
    Keep the summary readable, and concise.

    Output format:
    - Use only Arabic language.
    - JSON format only, with no additional text.
    - Limit to  8-14 sentences, ensuring the sentences captures all key points from the context.

    Use the JSON format as follows:
    {{
    "sentences": ["summary sentence 1", "summary sentence 2", "summary sentence 3", ...]
    }}

    <context>
    {text}
    </context>
    """
    return prompt


In [76]:
def questionsPrompt (text):     
    prompt = f"""
        You are a professional tool for generating multiple choice questions (MCQs) based strictly on the content provided in the <context> tag.
        Do not use any information or knowledge outside the context.
        Create exactly 3 MCQs with varying difficulty levels:
        - 1 easy question
        - 1 medium question
        - 1 hard question
        - The questions must be relevant to the information in the context.

        Output format:
        - Use only letters A, B, C, or D for the answer choices.
        - Use only Arabic language.
        - JSON format only, with no additional text, explanations, or instructions.
        - Follow this JSON structure exactly:
        [
        {
            "question": "السؤال",
            "A": "الخيار الأول",
            "B": "الخيار الثاني",
            "C": "الخيار الثالث",
            "D": "الخيار الرابع",
            "answer": " الاجابة الصحيحة (A, B, C, or D)"
        },
        ]
        <context>
        {text}
        </context>
        """
    
    return prompt


In [77]:
class ArabicSummarizationEvaluator:
    def __init__(self):
        self.rouge = Rouge()

        
    def preprocess_arabic_text(self, text: str) -> str:
        text = strip_tashkeel(text)
        text = strip_tatweel(text)
        text = re.sub(r'[\u064B-\u065F\u0670]', '', text)
        text = normalize.normalize_alef_ar(text)        
        text = normalize.normalize_teh_marbuta_ar(text)
        text = normalize.normalize_alef_maksura_ar(text)
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'@#$٪^&*[{}[\]""",]', '', text)
        text =simple_word_tokenize(text)
        text= ' '.join(text)
        return text
    
    def evaluate_summary(self, original_text: str, generated_summary: str) -> Dict:

        original_clean = self.preprocess_arabic_text(original_text)
        summary_clean = self.preprocess_arabic_text(generated_summary)
                
        evaluation_results = {
            'content_metrics': self._evaluate_content_preservation(original_clean, summary_clean),
            'linguistic_metrics': self._evaluate_linguistic_quality(summary_clean),
            'length_metrics': self._evaluate_length_metrics(original_clean, summary_clean),
            'readability_metrics': self._evaluate_arabic_readability(summary_clean)
        }

        return evaluation_results
    
    def _evaluate_content_preservation(self, original: str, summary: str) -> Dict:

        rouge_scores = self.rouge.get_scores(summary, original, avg=True)

        original_words = set(simple_word_tokenize(original))
        summary_words = set(simple_word_tokenize(summary))
        word_overlap =  len(original_words.intersection(summary_words)) / len(original_words)  if original_words else 0
        
        # Calculate key phrase preservation
        original_phrases = self._extract_key_phrases(original)
        summary_phrases = self._extract_key_phrases(summary)
        phrase_preservation = len(set(original_phrases).intersection(set(summary_phrases))) / len(original_phrases) if original_phrases else 0
        
        return {
            'rouge1_f1': rouge_scores['rouge-1']['f'],
            'rouge2_f1': rouge_scores['rouge-2']['f'],
            'rougeL_f1': rouge_scores['rouge-l']['f'],
            'word_overlap_ratio': word_overlap,
            'key_phrase_preservation': phrase_preservation
        }
    
    def _evaluate_linguistic_quality(self, text: str) -> Dict:
       
        words = simple_word_tokenize(text)
        sentences = self._split_arabic_sentences(text)
        avg_words_per_sentence = len(words) / len(sentences) if sentences else 0
        
        return {
            'avg_sentence_length': avg_words_per_sentence,
        }
    
    def _evaluate_length_metrics(self, original: str, summary: str) -> Dict:
        original_words = simple_word_tokenize(original)
        summary_words = simple_word_tokenize(summary)
        
        compression_ratio = len(summary_words) / len(original_words) if original_words else 0
        
        return {
            'compression ration': compression_ratio,
            'summary_length': len(summary_words),
            'original_length': len(original_words)
        }
    
    def _extract_key_phrases(self, text: str) -> List[str]:
        words = simple_word_tokenize(text)
        # Simple approach: consider sequences of 2-3 words as potential key phrases
        phrases = []
        for i in range(len(words)-1):
            phrases.append(' '.join(words[i:i+2]))
            if i < len(words)-2:
                phrases.append(' '.join(words[i:i+3]))
        return phrases
    
    def _split_arabic_sentences(self, text: str) -> List[str]:
        endings = ['。', '؟', '!', '．', '؛', ':', '.','،']
        sentences = []
        current = []
        
        for char in text:
            current.append(char)
            if char in endings:
                sentences.append(''.join(current))
                current = []
        
        if current: 
            sentences.append(''.join(current))
            
        return sentences
        
    def _evaluate_arabic_readability(self, text: str) -> Dict:

        sentences = self._split_arabic_sentences(text)
        words = simple_word_tokenize(text)
        
        avg_word_length = np.mean([len(word) for word in words]) if words else 0
        
        complex_words = [word for word in words if len(word) > 7]
        complexity_score = len(complex_words) / len(words)  if words else 0
        
        return {
            'AVG word length': avg_word_length,
            'complexity score': complexity_score,
            'sentences_count': len(sentences)
        }

In [78]:
def evaluate_summary(original_text ,summary):
    if __name__ == "__main__":
        original_text = original_text
        summary  = summary
        print("Generated Arabic summary using greedy:")
        print(summary)
        print("--------------------------------")
        evaluator = ArabicSummarizationEvaluator()
        results = evaluator.evaluate_summary(original_text, summary)
        # Print results
        import json
        print(json.dumps(results, ensure_ascii=False, indent=2))
        print("--------------------------------")
        print("Evaluation completed.")

In [79]:
def generate_summary(orginal_text):
    summary  = decoding_greedy.generate_text(prompt= summarizationPrompt(orginal_text), guardrails=False)
    return summary

def generate_questions(summary):
    questions = decoding_greedy.generate_text(prompt= questionsPrompt(summary), guardrails=False)
    return questions

In [80]:
summarized_text= generate_summary(preprocessedText4)

INFO:httpx:HTTP Request: POST https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-10-01 "HTTP/1.1 200 OK"
INFO:ibm_watsonx_ai.wml_resource:Successfully finished generate for url: 'https://eu-de.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-10-01'


In [81]:
evaluate_summary(preprocessedText4, summarized_text)

Generated Arabic summary using greedy:

    {
    "sentences": [
        "فضائل الخلفاء الراشدين تتضمن كونهم من أوائل من أسلموا وكونهم من العشرة المبشرين بالجنة وكونهم من أقوى الصحابة في إدارة شؤون المسلمين.",
        "تتضمن النشاط الطلابي تذكر تواريخ نهاية عهد كل خليفة: أبو بكر (عام 13 هـ)، عمر بن الخطاب (عام 23 هـ)، عثمان بن عفان (عام 35 هـ)، وعلي بن أبي طالب (عام 40 هـ).",
        "تميزت فترات حكم الخلفاء الراشدين بالعدل والمساواة وشهدت معارك ناجحة ومؤثرة ضد الروم والفرس.",
        "في عهد الخلفاء الراشدين، امتدت الفتوحات الإسلامية إلى الشام والعراق وفارس وشمال إفريقيا.",
        "شهدت المعارك البارزة مثل القادسية واليرموك ضد الفرس والروم انتصارات مهمة وتأثير كبير على التاريخ الإسلامي."
    ]
    } 
--------------------------------
{
  "content_metrics": {
    "rouge1_f1": 0.6625766821589071,
    "rouge2_f1": 0.3942307642867049,
    "rougeL_f1": 0.6257668662079867,
    "word_overlap_ratio": 0.7333333333333333,
    "key_phrase_preservation": 0.3283582089552239
  },
  "linguistic_metr

In [82]:
def prepareSummaryForQuestions(text):
    text = re.sub(r'@#$٪^&*[{}[\]""",]', '', text)
    text =simple_word_tokenize(text)
    text= ' '.join(text)
    return text

In [83]:
generate_questions(prepareSummaryForQuestions(summarized_text))

ValueError: Invalid format specifier ' "السؤال",
            "A": "الخيار الأول",
            "B": "الخيار الثاني",
            "C": "الخيار الثالث",
            "D": "الخيار الرابع",
            "answer": " الاجابة الصحيحة (A, B, C, or D)"
        ' for object of type 'str'

In [ ]:
import json
from typing import List, Dict, Union
import numpy as np
from collections import Counter
import time
import logging
from tqdm import tqdm

class ArabicMCQEvaluator:
    def __init__(self, max_questions: int = 1000):
        self.max_questions = max_questions
        self.arabic_stop_words = set(['من', 'إلى', 'عن', 'على', 'في', 'هذا', 'هذه', 'هو', 'هي', 'كان', 'كانت'])
        self.question_starters = ['ما', 'متى', 'أين', 'كيف', 'لماذا', 'من', 'هل']
        
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def validate_and_parse_input(self, questions: Union[str, List, Dict]) -> List[Dict]:
        """
        Validate and parse input questions into the correct format
        """
        try:
            # If input is a string, try to parse it as JSON
            if isinstance(questions, str):
                try:
                    questions = json.loads(questions)
                except json.JSONDecodeError:
                    raise ValueError("Invalid JSON string provided")

            # If input is a dictionary, wrap it in a list
            if isinstance(questions, dict):
                questions = [questions]

            # If input is not a list by now, it's invalid
            if not isinstance(questions, list):
                raise ValueError("Questions must be provided as a list, dictionary, or JSON string")

            # Validate each question
            valid_questions = []
            for i, question in enumerate(questions):
                if not isinstance(question, dict):
                    self.logger.warning(f"Skipping invalid question at index {i}: not a dictionary")
                    continue

                # Check required fields
                required_fields = ['question', 'A', 'B', 'C', 'D', 'answer']
                if not all(field in question for field in required_fields):
                    self.logger.warning(f"Skipping question at index {i}: missing required fields")
                    continue

                valid_questions.append(question)

            return valid_questions

        except Exception as e:
            self.logger.error(f"Error validating input: {str(e)}")
            raise

    def evaluate_questions(self, questions: Union[str, List, Dict]) -> Dict:
        """
        Evaluate a list of MCQ questions with input validation
        """
        try:
            # Validate and parse input
            valid_questions = self.validate_and_parse_input(questions)

            if not valid_questions:
                return {
                    'error': 'No valid questions found in input',
                    'overall_score': 0,
                    'detailed_scores': {},
                    'statistics': {'total_questions': 0},
                    'recommendations': ['يرجى التحقق من صحة تنسيق الأسئلة المدخلة']
                }

            if len(valid_questions) > self.max_questions:
                return {
                    'error': f'Too many questions. Maximum allowed: {self.max_questions}',
                    'overall_score': 0,
                    'detailed_scores': {},
                    'statistics': {'total_questions': len(valid_questions)},
                    'recommendations': [f'يرجى تقسيم الأسئلة إلى مجموعات أصغر (الحد الأقصى: {self.max_questions} سؤال)']
                }

            start_time = time.time()
            
            results = {
                'overall_score': 0,
                'detailed_scores': {},
                'statistics': {},
                'recommendations': [],
                'performance_metrics': {}
            }
            
            format_scores = []
            language_scores = []
            
            # Process questions with progress bar
            for i, question in tqdm(enumerate(valid_questions), total=len(valid_questions), desc="Evaluating questions"):
                question_number = i + 1
                question_scores = {}
                
                # Format and Completeness Check
                format_score = self._check_format(question)
                question_scores['format'] = format_score
                format_scores.append(format_score)
                
                # Language Quality Check
                lang_score = self._evaluate_language_quality(question)
                question_scores['language'] = lang_score
                language_scores.append(lang_score)
                
                # Options Quality Check
                options_score = self._evaluate_options(question)
                question_scores['options'] = options_score
                
                results['detailed_scores'][f'question_{question_number}'] = question_scores
            
            # Calculate statistics
            results['statistics'] = self._calculate_statistics(valid_questions)
            
            # Calculate overall score
            if format_scores and language_scores:
                results['overall_score'] = np.mean([
                    np.mean(format_scores),
                    np.mean(language_scores)
                ]) * 100
            
            # Generate recommendations
            results['recommendations'] = self._generate_recommendations(results)
            
            # Add performance metrics
            end_time = time.time()
            processing_time = end_time - start_time
            results['performance_metrics'] = {
                'total_processing_time_seconds': round(processing_time, 2),
                'average_time_per_question_seconds': round(processing_time / len(valid_questions), 4),
                'total_questions_processed': len(valid_questions),
                'invalid_questions_skipped': len(questions) - len(valid_questions) if isinstance(questions, list) else 0
            }
            
            return results

        except Exception as e:
            return {
                'error': f'Error processing questions: {str(e)}',
                'overall_score': 0,
                'detailed_scores': {},
                'statistics': {},
                'recommendations': ['حدث خطأ أثناء معالجة الأسئلة. يرجى التحقق من صحة التنسيق']
            }

    def _check_format(self, question: Dict) -> float:
        """Check the format and completeness of a question"""
        score = 1.0
        required_fields = ['question', 'A', 'B', 'C', 'D', 'answer']
        
        for field in required_fields:
            if field not in question or not isinstance(question[field], str) or not question[field].strip():
                score -= 0.2
                continue
        
        if 'answer' in question and question['answer'] not in ['A', 'B', 'C', 'D']:
            score -= 0.2
        
        return max(0, score)

    def _evaluate_language_quality(self, question: Dict) -> float:
        """Evaluate the linguistic quality of the question"""
        score = 1.0
        
        if not isinstance(question.get('question', ''), str):
            return 0.0

        if not any(question['question'].startswith(starter) for starter in self.question_starters):
            score -= 0.2
        
        if len(question['question'].split()) < 3:
            score -= 0.2
        
        if not question['question'].endswith('؟'):
            score -= 0.1
        
        return max(0, score)

    def _evaluate_options(self, question: Dict) -> float:
        """Evaluate the quality of answer options"""
        score = 1.0
        options = [question.get(opt, '') for opt in ['A', 'B', 'C', 'D']]
        
        if not all(isinstance(opt, str) for opt in options):
            return 0.0

        if len(set(options)) != len(options):
            score -= 0.3
        
        if any(len(opt.split()) < 1 for opt in options):
            score -= 0.2
        
        return max(0, score)

    def _calculate_statistics(self, questions: List[Dict]) -> Dict:
        """Calculate various statistics about the question set"""
        try:
            answer_distribution = Counter(q['answer'] for q in questions if isinstance(q.get('answer'), str))
            
            return {
                'total_questions': len(questions),
                'answer_distribution': dict(answer_distribution),
                'average_question_length': np.mean([len(q['question'].split()) for q in questions if isinstance(q.get('question'), str)]),
                'average_options_length': np.mean([
                    np.mean([len(q.get(opt, '').split()) for opt in ['A', 'B', 'C', 'D']])
                    for q in questions
                ])
            }
        except Exception as e:
            self.logger.error(f"Error calculating statistics: {str(e)}")
            return {
                'total_questions': len(questions),
                'error': str(e)
            }

    def _generate_recommendations(self, results: Dict) -> List[str]:
        """Generate recommendations based on evaluation results"""
        recommendations = []
        
        if results['overall_score'] < 70:
            recommendations.append("يجب تحسين جودة الأسئلة بشكل عام")
        
        
        return recommendations 

In [ ]:
evaluator = ArabicMCQEvaluator()  # Create an instance

questions_json = '''
[
{"question": "ما هي بعض المعارك التي خاضها الخلفاء الراشدين؟",
"A": "معركة اليرموك",
"B": "معركة ذات السلاسل",
"C": "معركة أجنادين",
"D": "جميع ما سبق",
"answer": ""   
},
{"question": "ما هو مبدأ أساسي قامت عليه الدولة الإسلامية في عهد الخلفاء الراشدين؟",
"A": "العدل والمساواة",        
"B": "القوة والغلبة",    
"C": "التفرقة والتمييز",
"D": "جميع ما سبق",
"answer": "A"
}

]

'''

results = evaluator.evaluate_questions(questions_json)
print(json.dumps(results, ensure_ascii=False, indent=2))

Evaluating questions: 100%|██████████| 2/2 [00:00<00:00, 12483.05it/s]

{
  "overall_score": 90.0,
  "detailed_scores": {
    "question_1": {
      "format": 0.6000000000000001,
      "language": 1.0,
      "options": 1.0
    },
    "question_2": {
      "format": 1.0,
      "language": 1.0,
      "options": 1.0
    }
  },
  "statistics": {
    "total_questions": 2,
    "answer_distribution": {
      "": 1,
      "A": 1
    },
    "average_question_length": 10.0,
    "average_options_length": 2.375
  },
  "recommendations": [],
  "performance_metrics": {
    "total_processing_time_seconds": 0.01,
    "average_time_per_question_seconds": 0.0066,
    "total_questions_processed": 2,
    "invalid_questions_skipped": 0
  }
}
